In [1]:
import pandas as pd
import os
import numpy as np
import math
from skimpy import clean_columns

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [2]:
year = 2018
df = clean_columns(pd.read_csv(os.path.join(os.getcwd(),'validation_results_'+str(year)+'.csv')))
df.head()

6 column names have been cleaned

,plotid,center_lon,center_lat,shape,size_m,sample_points,email,flagged,flagged_reason,collection_time,...,common_securewatch_date,total_securewatch_dates,pl_class,pl_which_raster,category_low_ndvi_impervious_surface,category_non_iceplant_vegetation,category_iceplant,category_water,validation_finished_yes_high_confidence,validation_finished_no_low_confidence
0,0,-119.742562,34.407042,square,20.0,1,brun@nceas.ucsb.edu,False,NaN,2022-11-15 04:43,...,NaN,0,0,2,0.0,100.0,0.0,0.0,100.0,0.0
1,1,-119.505269,34.384257,square,20.0,1,brun@nceas.ucsb.edu,False,NaN,2022-11-14 19:57,...,NaN,0,3,2,0.0,0.0,0.0,100.0,100.0,0.0
2,2,-119.639609,34.413251,square,20.0,1,brun@nceas.ucsb.edu,False,NaN,2022-11-14 19:57,...,NaN,0,3,2,0.0,0.0,0.0,100.0,100.0,0.0
3,3,-119.866794,34.409054,square,20.0,1,brun@nceas.ucsb.edu,False,NaN,2022-11-14 19:57,...,NaN,0,3,2,0.0,0.0,0.0,100.0,100.0,0.0
4,4,-120.488476,34.495967,square,20.0,1,brun@nceas.ucsb.edu,False,NaN,2022-11-14 19:59,...,NaN,0,2,0,100.0,0.0,0.0,0.0,100.0,0.0


In [3]:
def ref_class_column(df):

    map_class = df.pl_class
    ref_class = []

    for i in map_class.index:
        if df.category_non_iceplant_vegetation.loc[i] == 100:
            ref_class.append(0)
        elif df.category_iceplant.loc[i] == 100:
            ref_class.append(1)
        elif df.category_low_ndvi_impervious_surface.loc[i] == 100:
            ref_class.append(2)
        elif df.category_water.loc[i] == 100:
            ref_class.append(3)
        else:
            ref_class[j]= 100
            
    return ref_class

In [4]:
df['ref_class'] = ref_class_column(df)
df = df.drop(['center_lon', 'center_lat', 'shape', 'size_m', 'sample_points',
        'flagged', 'flagged_reason', 'collection_time', 
         'total_securewatch_dates', 'common_securewatch_date',
        'pl_which_raster', 
         'validation_finished_yes_high_confidence',
         'analysis_duration',
         'category_low_ndvi_impervious_surface','category_non_iceplant_vegetation',
         'category_iceplant', 'category_water',], axis =1)
df = df.rename( columns = {'pl_class':'map_class',
                       'validation_finished_no_low_confidence':'low_confidence'})
df

,plotid,email,map_class,low_confidence,ref_class
0,0,brun@nceas.ucsb.edu,0,0.0,0
1,1,brun@nceas.ucsb.edu,3,0.0,3
2,2,brun@nceas.ucsb.edu,3,0.0,3
3,3,brun@nceas.ucsb.edu,3,0.0,3
4,4,brun@nceas.ucsb.edu,2,0.0,2
...,...,...,...,...,...
445,445,brun@nceas.ucsb.edu,2,100.0,3
446,446,brun@nceas.ucsb.edu,2,0.0,2
447,447,brun@nceas.ucsb.edu,0,0.0,0
448,448,brun@nceas.ucsb.edu,1,0.0,0


In [5]:
np.unique(df.ref_class, return_counts=True)

(array([0, 1, 2, 3]), array([111,  58, 159, 122]))

In [6]:
np.unique(df.map_class, return_counts=True)

(array([0, 1, 2, 3]), array([100, 100, 150, 100]))

In [7]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# using confusion_matrix directly we get a matrix C such that
# C_{i,j} = known to be i, predicted as  j 
# The notation in the paper is 
# n_{i,j} = predicted as i, known to be j 
# so we need to take the transpose

n = confusion_matrix(df.ref_class, df.map_class, labels=range(0,4)).T
n

array([[ 63,   5,  30,   2],
       [ 36,  53,  11,   0],
       [ 12,   0, 116,  22],
       [  0,   0,   2,  98]])

In [8]:
pix_counts = pd.read_csv(os.path.join(os.getcwd(), 'rasters_'+str(year)+'_pixel_counts.csv'))
pix_counts

,n_nonice_2018,n_ice_2018,n_ground_2018,n_water_2018,raster
0,35863561,5897584,113626707,59950901,LS_merged_crs26910_S_2018
1,6730191,899027,24887840,16301218,LS_merged_crs26910_W_2018
2,79013822,3880728,73808926,66038443,LS_merged_crs26911_2018


In [9]:
total_pix = sum([sum(pix_counts.n_nonice_2018),
                  sum(pix_counts.n_ice_2018),
                  sum(pix_counts.n_ground_2018),
                  sum(pix_counts.n_water_2018)])

In [10]:
W = []      # proportion of area mapped as class i
n_idot = [] # pixels in sample that had class i in map (predicted as i, any true class j)
U_hat = []  # estimated users' accuracy (precision for each class: TP/(TP+FP))

for i in range(0,4):
    W.append( sum(pix_counts.iloc[:,i]) / total_pix)
    n_idot.append(sum(n[i,:]))
    U_hat.append(n[i,i] / n_idot[i])

In [12]:
O = sum([W[i]*n[i,i]/n_idot[i] for i in range(0,4)])
print('overall accuracy:', O*100)

var_O = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)])
# std error of estimated overall accuracy -- paper equation (5)
print('overall accuracy std error:', np.sqrt(var_O)*100, '\n')

print('users accuracy:', U_hat)

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)]
print('users accuracies std errors:', np.sqrt(var_U_hat)*100)

overall accuracy: 79.25943006350468
overall accuracy std error: 1.9715593227663106 

users accuracy: [0.63, 0.53, 0.7733333333333333, 0.98]
users accuracies std errors: [4.85236587 5.01613558 3.42992055 1.40705294]


In [14]:
p_dotk_hat = []
P_hat = []  # estimated producer's accurace (sensitiviy for each class TP/(TP+FN))

for k in range(0,4):
    partial = [ W[i]*n[i,k]/n_idot[i] for i in range(0,4) ]
    p_dotk_hat.append( sum(partial))  # equation (9)
p_dotk_hat

for i in range(0,4):
    P_hat.append( (W[i]*n[i,i]/n_idot[i]) / p_dotk_hat[i])

print('producers accuracy:', P_hat)

producers accuracy: [0.7862357626600711, 0.48205231249592345, 0.8021362603845338, 0.8059566637712716]


In [ ]:
high_confidence = df[df.low_confidence == 0]
high_confidence

In [ ]:
n = confusion_matrix(high_confidence.ref_class, high_confidence.map_class, labels=range(0,4)).T
n

In [ ]:
n_idot = [] # pixels in sample that had class i in map (predicted as i, any true class j)
U_hat = []  # estimated users' accuracy (precision for each class: TP/(TP+FP))

for i in range(0,4):
    n_idot.append(sum(n[i,:]))
    U_hat.append(n[i,i] / n_idot[i])

O = sum([W[i]*n[i,i]/n_idot[i] for i in range(0,4)])
print('overall accuracy:', O*100)

var_O = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)])
# std error of estimated overall accuracy -- paper equation (5)
print('overall accuracy std error:', np.sqrt(var_O)*100, '\n')

print('users accuracy:', U_hat)

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1)for i in range(0,4)]
var_U_hat
print('users accuracies std errors:', np.sqrt(var_U_hat)*100)

In [ ]:
196*np.sqrt(var_U_hat)

In [ ]:
np.sqrt(var_O)*196